In [1]:
import pandas as pd
import numpy as np

In [2]:
# 1、读取数据
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
column_name = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                   'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
                   'Normal Nucleoli', 'Mitoses', 'Class']

data = pd.read_csv(path, names=column_name)

In [5]:
data.head()


,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [10]:
# 2、缺失值处理
# 1）替换——》np.nan
data = data.replace(to_replace="?",value=np.nan)
# 2)删除缺失样本
data.dropna(inplace=True)

In [12]:
data.isnull().any()#不存在缺失值

Sample code number             False
Clump Thickness                False
Uniformity of Cell Size        False
Uniformity of Cell Shape       False
Marginal Adhesion              False
Single Epithelial Cell Size    False
Bare Nuclei                    False
Bland Chromatin                False
Normal Nucleoli                False
Mitoses                        False
Class                          False
dtype: bool

In [13]:
# 3、划分数据集
from sklearn.model_selection import train_test_split

In [14]:
# 筛选特征值和目标值
x = data.iloc[:,1:-1]
y = data["Class"]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [18]:
# 3.特征工程
from sklearn.preprocessing import StandardScaler
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [19]:
x_train

array([[-1.19730591, -0.70485363, -0.75922851, ..., -0.59167282,
        -0.61533154,  0.21944185],
       [ 1.89182378,  0.27670138,  0.24092157, ...,  0.64817636,
         0.68250471, -0.35970365],
       [-1.19730591, -0.70485363, -0.09246179, ..., -1.00495587,
        -0.61533154, -0.35970365],
       ...,
       [-0.51083264,  0.27670138,  0.24092157, ..., -0.17838976,
         0.03358658, -0.35970365],
       [ 0.51887725,  2.23981141,  0.57430493, ...,  1.06145942,
         2.30480003, -0.35970365],
       [ 1.20535052,  0.27670138,  0.24092157, ..., -0.17838976,
         0.03358658, -0.35970365]], shape=(512, 9))

In [20]:
from sklearn.linear_model import LogisticRegression

In [22]:
estimator = LogisticRegression()
estimator.fit(x_train,y_train)

LogisticRegression()

In [23]:
# 逻辑回归的模型参数：回归系数和偏置
estimator.coef_

array([[ 1.29989263,  0.39473928,  0.79577127,  0.65767952, -0.09727041,
         1.59989409,  0.87698755,  1.06608615,  0.66680494]])

In [24]:
estimator.intercept_

array([-0.67145436])

In [25]:
# 6、模型评估
# 方法1：直接比对真实值和预测值
y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
print("直接比对真实值和预测值:\n", y_test == y_predict)

# 方法2：计算准确率
score = estimator.score(x_test, y_test)
print("准确率为：\n", score)

y_predict:
 [2 4 2 2 2 4 2 2 4 2 2 4 4 4 2 2 4 4 2 4 4 4 2 4 2 4 2 4 2 2 2 2 4 2 4 2 2
 2 2 4 2 4 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2 2 2 2 4 4 4 4 2 2 2 2 2 2 4 2 4
 2 2 2 2 2 4 4 2 2 2 2 2 2 4 2 2 4 4 4 2 4 2 2 2 4 4 2 2 2 4 4 4 2 2 4 2 4
 2 2 4 2 2 2 4 2 2 2 2 4 2 2 2 2 2 2 2 4 2 2 2 2 2 4 4 2 2 2 2 2 2 2 2 4 2
 2 2 2 4 4 2 2 2 4 4 2 4 4 2 2 2 2 2 4 2 4 4 2]
直接比对真实值和预测值:
 485    True
14     True
392    True
16     True
171    True
       ... 
117    True
327    True
53     True
328    True
280    True
Name: Class, Length: 171, dtype: bool
准确率为：
 0.9707602339181286


In [28]:
 # 查看精确率、召回率、F1-score
from sklearn.metrics import classification_report
report = classification_report(y_test,y_predict,labels=[2,4],target_names=["良性","恶性"])

In [29]:
print(report)

              precision    recall  f1-score   support

          良性       0.99      0.97      0.98       118
          恶性       0.93      0.98      0.95        53

    accuracy                           0.97       171
   macro avg       0.96      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171



In [32]:
y_true = np.where(y_test > 3,1,0)

In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true,y_predict)

np.float64(0.9736168851934761)